# Descarga de imagenes de Satelite Sentinel2

In [1]:
import ee

In [1]:
#ee.Authenticate()

In [3]:
import ee
import geemap
import pandas as pd
ee.Initialize()

In [4]:
#!pip install jupyter notebook

In [5]:
Map = geemap.Map(basemap='Esri.WorldImagery')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [6]:
zona = ee.FeatureCollection("projects/ee-geomatica/assets/EEA_INIA/Zona_EEA_Geo")

In [7]:
# Extraer la lista de paises
lista_EEA = zona.reduceColumns(ee.Reducer.toList(),["EEA"]).get("list").getInfo()
print(lista_EEA)

['DONOSO', 'EL PORVENIR', 'PUCALLPA', 'SANTA ANA']


In [8]:
Selec_EEA = lista_EEA[3]
Selec_EEA

'SANTA ANA'

In [9]:
EEA = zona.filterMetadata('EEA', 'equals', Selec_EEA)

In [10]:
geometria = EEA.geometry()

In [11]:
# Visualizar en mapa
Map.center_object(EEA,13)
Map.addLayer(EEA, {"color" : "green"}, "EEA_" + Selec_EEA)
Map

Map(bottom=812.0, center=[-12.012035762649006, -75.2214429775274], controls=(WidgetControl(options=['position'…

In [12]:
# Coleccion de Sentinel-2 SR
S2_RS = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
          .filterDate('2022-04-01','2022-12-31')\
          .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',10)\
          .filterBounds(EEA)\
         .sort('CLOUDY_PIXEL_PERCENTAGE', True)

In [13]:
# Imprimir la lista de ID Imagenes
ID_S2 = S2_RS.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()

In [14]:
for i in ID_S2:
    print(i)

20220729T151709_20220729T152454_T18LVM
20220614T151711_20220614T152004_T18LVM
20220629T151709_20220629T151709_T18LVM
20220818T151709_20220818T152433_T18LVM
20220813T151721_20220813T152335_T18LVM
20220609T151709_20220609T151713_T18LVM
20220823T151721_20220823T152332_T18LVM
20220828T151709_20220828T152330_T18LVM
20220902T151721_20220902T152331_T18LVM
20220724T151711_20220724T152452_T18LVM
20220714T151721_20220714T151723_T18LVM
20220530T151709_20220530T151704_T18LVM
20221201T151701_20221201T152033_T18LVM
20220803T151721_20220803T151715_T18LVM
20221022T151711_20221022T152231_T18LVM
20220912T151711_20220912T152434_T18LVM
20221012T151711_20221012T151922_T18LVM
20220907T151709_20220907T151918_T18LVM
20220520T151709_20220520T151704_T18LVM
20221111T151711_20221111T151707_T18LVM
20220619T151709_20220619T151708_T18LVM


In [15]:
len(ID_S2)

21

In [16]:
Fecha_inicio = '2022-04-01'
Fecha_final = '2022-12-31'
Por_Nubosidad = 10

In [ ]:
for a in range(len(lista_EEA)):
    EEA = zona.filterMetadata('EEA', 'equals', lista_EEA[a])
    geometria = EEA.geometry()
    # Coleccion de Sentinel-2 SR
    S2_RS = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
                .filterDate(Fecha_inicio,Fecha_final)\
                .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',Por_Nubosidad)\
                .filterBounds(EEA)\
                .sort('CLOUDY_PIXEL_PERCENTAGE', True)
    # Imprimir la lista de ID Imagenes
    ID_S2 = S2_RS.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
    # Proceso de image
    for i in range(len(ID_S2)):
        # Mantiene la convención de nombre de archivo original    
        image = ee.Image("COPERNICUS/S2_SR_HARMONIZED" + "/" + ID_S2[i])
        bandas_RS = ["B1","B2","B3","B4","B5","B6","B7","B8","B9","B11","B12"]
        # Seleccionar imagenes por bandas
        img_S2_RS = image.select(bandas_RS)
        # Escalar la imagen Sentinel2 RS 0 - 1
        img_S2_RS_es = img_S2_RS.multiply(0.0001)
        # Reproyectar segun zona de estudio
        img_S2_RS_es = img_S2_RS_es.reproject(crs="EPSG:32718", scale=10)
        # Recortar RS
        img_S2_RS_es_clip = img_S2_RS_es.clip(EEA)
        # Exportar en google drive
        #geemap.ee_export_image_to_drive(img_S2_RS_es_clip, 
        #                                description="S2_" + ID_S2[i][0:15]+"_RS",
        #                                folder='GEE_Sentinel2', 
        #                                region=geometria, 
        #                                scale=10)
        # Descarga link descarga
        link_URL = img_S2_RS_es_clip.getDownloadURL({
            'name': lista_EEA[a] + "_S2_" + ID_S2[i][0:15]+"_RS",
            'bands': bandas_RS,
            'region': geometria,
            'scale': 10,
            'filePerBand': False #  True = bandas separadas
        })
        print('Archivo zip ' + lista_EEA[a] + "_S2_" + ID_S2[i][0:15]+"_RS", link_URL)
    

Archivo zip DONOSO_S2_20220503T152639_RS https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/4e5724280cbf829c5320066e1f99ad74-c9609248c4bcd9a63f1ffa3a6bde04be:getPixels
Archivo zip DONOSO_S2_20220503T152639_RS https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/d40a9d24f122fb42a4b5ff008413dc62-2948a035382745827094e853ea0cf5b7:getPixels
Archivo zip DONOSO_S2_20220528T152651_RS https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/ead725a4115420a8c5652a1c6410262b-0f914e499c3cfdcbfdc7502cccdc7b07:getPixels
Archivo zip DONOSO_S2_20221214T152641_RS https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/45243a2211c388ad5f41a2f6c80bd932-d4ad13a8a04a74ca2410e9c93514b953:getPixels
Archivo zip EL PORVENIR_S2_20220826T152651_RS https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/49dd33b1c0dd4a7c6143992e7ca582c5-cd333a5bc0de761d98c6d3ba20ecac3a:getPixels
Archivo zip EL PORVENIR_S